### 패키지 및 데이터
필요한 라이브러리 임포트 및 데이터 파일 읽어오기

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

### 컬럼명 변경
'도보 10분거리 내 지하철역 수(환승노선 수 반영)' -> 지하철

'도보 10분거리 내 버스정류장 수' -> 버스

In [ ]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

### 결측치 확인

In [ ]:
train.isna().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       549
임대료         549
지하철         207
버스            0
단지내주차면수       0
등록차량수         0
dtype: int64

In [ ]:
test.isna().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64

자격유형 결측치 경우 아래 행을 참조해 채우도록 설정

In [ ]:
train['자격유형'] = train['자격유형'].fillna(method='backfill')
test['자격유형'] = test['자격유형'].fillna(method='backfill')

임대보증금과 임대료는 -를 null로 바꾼 뒤 float 타입으로 변경하고 결측치는 1로 채움

In [ ]:
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(1)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(1)

### 공급유형, 자격유형 그룹화
바페르 님의 공유 코드를 참고 해 그룹화 작업을 진행했습니다.
https://dacon.io/competitions/official/235745/codeshare/2851?page=1&dtype=recent

In [ ]:
train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

### 문자형 데이터들 숫자로 매핑

In [ ]:
local_map={}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc]=i
#'경상북도': 0, '경상남도': 1, '대전광역시': 2, '경기도': 3, '전라북도': 4, '강원도': 5, '광주광역시': 6, 
# '충청남도': 7, '부산광역시': 8, '제주특별자치도': 9, '울산광역시': 10, '충청북도': 11, '전라남도': 12, '대구광역시': 13, 
# '서울특별시': 14, '세종특별자치시': 15

case_map={}
for i, case in enumerate(train['임대건물구분'].unique()):
    case_map[case] = i
#'아파트': 0, '상가': 1

supply_map={}
for i, stype in enumerate(train['공급유형'].unique()):
    supply_map[stype] =i
#'국민임대': 0, '공공임대(50년)': 1, '영구임대': 2, '임대상가': 3, '공공임대(10년)': 4, 
# '공공임대(분납)': 5, '장기전세': 6, '공공분양': 7, '행복주택': 8, '공공임대(5년)': 9
# ---> 수정됨
#'국민임대/장기전세': 0, '공공임대(50년)': 1, '영구임대': 2, '임대상가': 3, '공공임대(5년/10년/분납/분양)': 4, '행복주택': 5

type_map={}
for i, s_type in enumerate(train['자격유형'].unique()):
    type_map[s_type] = i


train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

train['임대건물구분'] = train['임대건물구분'].map(case_map)
test['임대건물구분'] = test['임대건물구분'].map(case_map)

train['공급유형'] = train['공급유형'].map(supply_map)
test['공급유형'] = test['공급유형'].map(supply_map)

train['자격유형'] = train['자격유형'].map(type_map)
test['자격유형'] = test['자격유형'].map(type_map)

### 파생변수 생성

전체면적합 : 단지 내 전용면적의 전체 합

건물가치 : 단지 내 임대 보증금의 전체 합

평균임대 : 단지 내 한 세대에 대한 평균 임대 보증금

55미만/이상 : 전용면적을 55m^2를 기준으로 해 세대수를 따로 계산함

아파트, 상가 : 아파트와 상가가 함께 있는 주상복합 단지를 가려내기 위해, 아파트 상가에 대한 변수도 따로 생성


In [ ]:
total_area = train['전용면적']*train['전용면적별세대수']
train['전체면적합'] = total_area

total_area = test['전용면적']*test['전용면적별세대수']
test['전체면적합'] = total_area

total_fee = train['임대보증금'] * train['전용면적별세대수']
train['건물가치'] = total_fee
average_fee = total_fee / train['총세대수']
train['평균임대'] = average_fee

total_fee = test['임대보증금'] * test['전용면적별세대수']
test['건물가치'] = total_fee
average_fee = total_fee / test['총세대수']
test['평균임대'] = average_fee

train['is_55미만'] = np.where(train['전용면적']<55,1,0)
train['is_55이상'] = np.where(train['전용면적']>=55,1,0)
test['is_55미만'] = np.where(test['전용면적']<55,1,0)
test['is_55이상'] = np.where(test['전용면적']>=55,1,0)

train['temp_55미만'] = train['is_55미만']*train['전용면적별세대수']
train['temp_55이상'] = train['is_55이상']*train['전용면적별세대수']
test['temp_55미만'] = test['is_55미만']*test['전용면적별세대수']
test['temp_55이상'] = test['is_55이상']*test['전용면적별세대수']

train['아파트'] = np.where(train['임대건물구분']==0, 1, 0)
train['상가'] = np.where(train['임대건물구분']==1, 1, 0)
test['아파트'] = np.where(test['임대건물구분']==0, 1, 0)
test['상가'] = np.where(test['임대건물구분']==1, 1, 0)

### 전용면적 수정 및 상하한 적용

전용면적을 5의 배수의 값을 가지도록 재설정하고, 상하한값을 적용

In [ ]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

### 같은 코드의 단지끼리 데이터 취합


In [ ]:
new_total_area=[] # 전체 면적합 저장
change_index=[0] # 단지코드가 변화하는 인덱스를 저장
new_total_fee=[] # 전체 임대료 저장
new_average_fee=[] # 평균 임대료 저장
is_apt=[]
is_store=[]
over_55=[]
lower_55=[]
change_index_number = 0

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0]) #shape[0] : 행의 개수, shape[1] : 열의 개수

    number = temp.shape[0]

    change_index_number += number
    change_index.append(change_index_number)

    lastvalue_area = 0
    lastvalue_fee = 0
    lastvalue_avgfee =0
    lastvalue_over55=0
    lastvalue_lower55=0
    aptvalue=0
    storevalue=0
    for j in range(number): # 같은 단지 코드내에서 연산 돌아가는 루프
        lastvalue_area +=train.iloc[change_index[i]+j]['전체면적합'] # 같은 코드의 전용면적*세대수 총 합
        lastvalue_fee += train.iloc[change_index[i]+j]['건물가치']
        lastvalue_avgfee += train.iloc[change_index[i]+j]['평균임대']
        lastvalue_over55 += train.iloc[change_index[i]+j]['temp_55이상']
        lastvalue_lower55 += train.iloc[change_index[i]+j]['temp_55미만']
        aptvalue +=train.iloc[change_index[i]+j]['아파트']
        storevalue += train.iloc[change_index[i]+j]['상가']

    new_total_area.append(lastvalue_area)
    new_total_fee.append(lastvalue_fee)
    new_average_fee.append(lastvalue_avgfee)
    over_55.append(lastvalue_over55)
    lower_55.append(lastvalue_lower55)
    if aptvalue != 0:
        is_apt.append(1)
    else:
        is_apt.append(aptvalue)
    if storevalue != 0:
        is_store.append(1)
    else:
        is_store.append(storevalue)

test_new_total_area=[] # 전체 면적합 저장
test_new_total_fee=[] # 전체 임대료 저장
test_new_average_fee=[] # 평균 임대료 저장
test_change_index=[0] # 단지코드가 변화하는 인덱스를 저장
test_over_55=[]
test_lower_55=[]
test_is_apt=[]
test_is_store=[]
test_change_index_number = 0

for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0]) #shape[0] : 행의 개수, shape[1] : 열의 개수

    number = temp.shape[0]

    test_change_index_number += number
    test_change_index.append(test_change_index_number)

    lastvalue_area = 0
    lastvalue_fee = 0
    lastvalue_avgfee =0
    lastvalue_over55=0
    lastvalue_lower55=0
    aptvalue=0
    storevalue=0
    for j in range(number): # 같은 단지 코드내에서 연산 돌아가는 루프
        lastvalue_area +=test.iloc[test_change_index[i]+j]['전체면적합'] # 같은 코드의 전용면적*세대수 총 합
        lastvalue_fee += test.iloc[test_change_index[i]+j]['건물가치']
        lastvalue_avgfee += test.iloc[test_change_index[i]+j]['평균임대']
        lastvalue_over55 += test.iloc[test_change_index[i]+j]['temp_55이상']
        lastvalue_lower55 += test.iloc[test_change_index[i]+j]['temp_55미만']
        aptvalue +=test.iloc[test_change_index[i]+j]['아파트']
        storevalue += test.iloc[test_change_index[i]+j]['상가']

    test_new_total_area.append(lastvalue_area)
    test_new_total_fee.append(lastvalue_fee)
    test_new_average_fee.append(lastvalue_avgfee)
    test_over_55.append(lastvalue_over55)
    test_lower_55.append(lastvalue_lower55)
    if aptvalue != 0:
        test_is_apt.append(1)
    else:
        test_is_apt.append(aptvalue)
    if storevalue != 0:
        test_is_store.append(1)
    else:
        test_is_store.append(storevalue)
    
new_train = pd.DataFrame()
new_test = pd.DataFrame()

columns = ['단지코드','총세대수','공급유형','자격유형','공가수', '지역', '지하철','버스','단지내주차면수']
target = '등록차량수'

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    #for col in area_columns:
    #    area = float(col.split('_')[-1])
    #    new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]

new_train.insert(2,'단지전체면적', new_total_area)
new_test.insert(2,'단지전체면적', test_new_total_area)

new_train.insert(3,'총임대보증금', new_total_fee)
new_test.insert(3,'총임대보증금', test_new_total_fee)

new_train.insert(4,'평균임대보증금', np.log(new_average_fee))
new_test.insert(4,'평균임대보증금', np.log(test_new_average_fee))

new_train.insert(5,'55미만세대', lower_55)
new_test.insert(5,'55미만세대', test_lower_55)

new_train.insert(6,'55이상세대', over_55)
new_test.insert(6,'55이상세대', test_over_55)

new_train.insert(7, '임대건물_아파트', is_apt)
new_test.insert(7, '임대건물_아파트', test_is_apt)

new_train.insert(8, '임대건물_상가', is_store)
new_test.insert(8, '임대건물_상가', test_is_store)

411it [00:04, 86.20it/s]
150it [00:01, 86.69it/s]
411it [00:02, 159.24it/s]
150it [00:00, 165.88it/s]


### 지하철, 버스 결측치 0으로 대체

In [ ]:
cols = ['지하철','버스']
for col in cols:
    mf = new_train[col].value_counts().idxmax()
    new_train[col].fillna(mf,inplace=True)

cols = ['지하철','버스']
for col in cols:
    mf = new_test[col].value_counts().idxmax()
    new_test[col].fillna(mf,inplace=True)

### 학습을 위한 축 값 설정

In [ ]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

### 학습 모델 파라미터 선정
GS.best_params_ 의 결과값에 따라 학습 모델 파라메터 선정 됨

In [ ]:
from sklearn.metrics import make_scorer
def rmse(p,a):
    difference = p-a
    squared = difference ** 2
    mean = squared.mean()
    score = np.sqrt(mean)
    return score
rmse_scorer = make_scorer(rmse, greater_is_better=False)
rmse_scorer

from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

from sklearn.model_selection import GridSearchCV
parameters = {
    'max_depth':[1,2,3,4,5,6,7],
    'random_state':[10,30,50,70,90],
}
GS = GridSearchCV(model, param_grid=parameters, cv=20, scoring=rmse_scorer)
GS.fit(x_train,y_train)

GS.best_params_

{'max_depth': 4, 'random_state': 90}

### 추론


In [ ]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

model_Lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.1, random_state=20))
model_ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.1, l1_ratio=0.7, random_state=20))
model_GBoost = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.05, max_depth=4,max_features='sqrt', min_samples_leaf=10,min_samples_split=5,loss='huber',random_state=90)

model_Lasso.fit(x_train, y_train)
Lasso_predictions=model_Lasso.predict(x_test)
train_Lasso=model_Lasso.predict(x_train)

model_ENet.fit(x_train, y_train)
ENet_predictions=model_ENet.predict(x_test)
train_ENet=model_ENet.predict(x_train)

model_GBoost.fit(x_train, y_train)
GBoost_predictions=model_GBoost.predict(x_test)
train_GBoost=model_GBoost.predict(x_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25489.322915440425, tolerance: 6123.326607785889
  positive)


### 결과 파일 생성

In [ ]:
predictions=(Lasso_predictions + ENet_predictions + GBoost_predictions) / 3

submission['num'] = predictions
submission.to_csv('include_supply_type_3_model.csv', index=False)